### Raniery Mendes
### Playing with CNNs and Data Augmentation

In [1]:
new_base_dir = "./splitted_data"
original_dir = "./train"

In [2]:
#function to organize dataset in sets 
import os, shutil, pathlib
def make_subset(subset_name, start_index, end_index): 
    for category in ("cat", "dog"):
            dir = f"{new_base_dir}/{subset_name}/{category}/"
            os.makedirs(dir)
            fnames = [f"{category}.{i}.jpg" for i in range(start_index, end_index)] 
            for fname in fnames:
                shutil.copyfile(src=f"{original_dir}/{fname}",
                                            dst=f"{dir}/{fname}")


In [3]:
#create training, test, and validation sets.                           
make_subset("train", start_index=0, end_index=1000)
make_subset("validation", start_index=1000, end_index=1500)
make_subset("test", start_index=1500, end_index=2500)

In [7]:
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.utils import image_dataset_from_directory

In [26]:
#set up model 
model = keras.Sequential(name="practicing-cnns")
model.add(keras.Input(shape=(180,180,3)))
model.add(layers.Rescaling(1./255))
model.add(layers.Conv2D(filters=32, kernel_size=3, activation='relu'))
model.add(layers.MaxPooling2D(pool_size=2))
model.add(layers.Conv2D(filters=64, kernel_size=3, activation='relu'))
model.add(layers.MaxPooling2D(pool_size=2))
model.add(layers.Conv2D(filters=128, kernel_size=3, activation='relu'))
model.add(layers.MaxPooling2D(pool_size=2))
model.add(layers.Conv2D(filters=256, kernel_size=3, activation='relu'))
model.add(layers.MaxPooling2D(pool_size=2))
model.add(layers.Conv2D(filters=256, kernel_size=3, activation='relu'))
model.add(layers.Flatten())
model.add(layers.Dense(1,activation='sigmoid'))





In [27]:

train_dataset = image_dataset_from_directory(f"{new_base_dir}/train", image_size=(180, 180), batch_size=32)
validation_dataset = image_dataset_from_directory(f"{new_base_dir}/validation", image_size=(180, 180),batch_size=32)
test_dataset = image_dataset_from_directory(f"{new_base_dir}/test",image_size=(180, 180), batch_size=32)

Found 2000 files belonging to 2 classes.
Found 1000 files belonging to 2 classes.
Found 2000 files belonging to 2 classes.


In [28]:
#check all data loaders provide same batch size as expected. 

print("Train")
for batch_data, batch_label in train_dataset:
    print("Data ",batch_data.shape)
    print("Labels ", batch_label.shape)
    break;

print("Test")
for batch_data, batch_label in test_dataset:
    print("Data ",batch_data.shape)
    print("Labels ", batch_label.shape)
    break;
    
print("Validation")
for batch_data, batch_label in validation_dataset:
    print("Data ",batch_data.shape)
    print("Labels ", batch_label.shape)
    break;

Train
Data  (32, 180, 180, 3)
Labels  (32,)
Test
Data  (32, 180, 180, 3)
Labels  (32,)
Validation
Data  (32, 180, 180, 3)
Labels  (32,)


In [29]:
#set up callback so we can monitor the model's performance
#this allow us to save the best model, i.e. save only when we decrease the val loss. 

callbacks = [keras.callbacks.ModelCheckpoint(filepath="my_cnn_model.keras", save_best_only=True, monitor='val_loss')]

In [32]:
#Verify model parameters and set optimizer
model.summary()
model.compile(loss="binary_crossentropy", metrics=["accuracy"], optimizer="rmsprop")

Model: "practicing-cnns"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ rescaling_2 (Rescaling)         │ (None, 180, 180, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_10 (Conv2D)              │ (None, 178, 178, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_8 (MaxPooling2D)  │ (None, 89, 89, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_11 (Conv2D)              │ (None, 87, 87, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_9 (MaxPooling2D)  │ (None, 43, 43, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_12 (Conv2D)              │ (None, 41, 41, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_10 (MaxPooling2D) │ (None, 20, 20, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_13 (Conv2D)              │ (None, 18, 18, 256)    │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_11 (MaxPooling2D) │ (None, 9, 9, 256)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_14 (Conv2D)              │ (None, 7, 7, 256)      │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_2 (Flatten)             │ (None, 12544)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1)              │        12,545 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 991,043 (3.78 MB)

 Trainable params: 991,041 (3.78 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 2 (12.00 B)

In [33]:
#fit the CNN model

history = model.fit(train_dataset,epochs=30,validation_data=validation_dataset, callbacks=callbacks)

Epoch 1/30


ValueError: 'practicing cnns and DA_practicing-cnns_conv2d_10_kernel_velocity' is not a valid scope name. A scope name has to match the following pattern: ^[A-Za-z0-9_.\\/>-]*$